In [1]:
import os
import sys
import datetime
import numpy as np
import pandas as pd
from numpy import random as npr

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
SEED = 2021
MOVIELENS_DATASET_PATH = "data.csv"
REAL_DATASET_SAMPLE_SIZE = 0.1
CACHE_DIR = "./cache"
N_CHUNKS = 2
npr.seed(SEED)

In [16]:
real_dataset = pd.read_csv(MOVIELENS_DATASET_PATH)
npr.seed(SEED)
real_dataset = real_dataset.sample(frac=REAL_DATASET_SAMPLE_SIZE)
real_dataset["user_id"] = real_dataset["user_id"].astype('category').cat.codes
real_dataset["movie_id"] = real_dataset["movie_id"].astype('category').cat.codes
real_dataset["rating"] = real_dataset["rating"]
real_dataset["timestamp"] = pd.to_datetime(real_dataset["timestamp"], unit="s")
real_dataset = real_dataset.sort_values("timestamp")
real_dataset

/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,movie_id,rating,timestamp,movie_name,tag0,tag1,tag2,tag3,tag4,...,tag12,tag13,tag14,tag15,tag16,tag17,gender,age,occupation,zip
456961,5956,527,4,2000-04-25 23:07:36,"Ciao, Professore! (Io speriamo che me la cavo ...",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,M,25,6,11106
456754,5956,754,5,2000-04-25 23:09:51,Casablanca (1942),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,M,25,6,11106
456632,5956,761,5,2000-04-25 23:09:51,"Wizard of Oz, The (1939)",0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,M,25,6,11106
456920,5956,296,4,2000-04-25 23:15:19,To Live (Huozhe) (1994),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,M,25,6,11106
456753,5956,16,3,2000-04-25 23:16:24,Sense and Sensibility (1995),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,M,25,6,11106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593003,5235,1041,4,2003-02-27 04:30:55,Bad Taste (1987),0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,M,25,1,10463
449048,5867,2595,1,2003-02-27 17:44:59,Adventures of Buckaroo Bonzai Across the 8th D...,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,M,25,4,19713
448957,5867,1136,4,2003-02-27 17:53:49,Sling Blade (1996),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,M,25,4,19713
449196,5867,2800,3,2003-02-27 18:04:50,Ghost Dog: The Way of the Samurai (1999),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,M,25,4,19713


In [17]:
def to_rating_matrix(long_table_df):
    matrix = long_table_df[["user_id", "movie_id", "rating"]]
    matrix["item_id"] = matrix["movie_id"].astype('category').cat.codes
    matrix["user_id"] = matrix["user_id"].astype('category').cat.codes
    return matrix.pivot(index="user_id", columns="item_id", values="rating").fillna(0)

In [18]:
def evaluate(rating_matrix):
    evaluators = [
        evaluator.TrainTestExecutorConfig(
            factory=trainers.AutoRecTrainTestExecutor,
            args={"config": {"epoch": 50}},
            model_name="autorec"
        ),
        evaluator.TrainTestExecutorConfig(
            factory=trainers.SvdTrainTestExecutor,
            args={},
            model_name="svd"
        ),
        evaluator.TrainTestExecutorConfig(
            factory=trainers.KnnTrainTestExecutor,
            args={},
            model_name="knn"
        )
    ]

    start_time = datetime.datetime.utcnow()
    errors = []
    for evl in evaluators:
        executor = evl.factory(**evl.args)
        error = executor(rating_matrix, 0.1, 1.0)
        errors.append(error)
    end_time = datetime.datetime.utcnow()
    return errors, end_time - start_time

In [19]:
import importlib
from modules import models, evaluator, trainers, utils
importlib.reload(models)
importlib.reload(evaluator)
importlib.reload(trainers)

<module 'modules.trainers' from '/Users/vldpro/Workspace/university/recsys/modules/trainers.py'>

In [ ]:
%%time
npr.seed(SEED)
parts = np.array_split(real_dataset, N_CHUNKS)
errors_list = []
for i, part in enumerate(parts):
    matrix = to_rating_matrix(part)
    errors, duration = evaluate(matrix)
    print(f"Part {i} processed in {duration}")
    errors_list.append({"errors": errors, "part": i})

/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Load data finished. Number of users: 3196 Number of items: 3035
IAutoRec.
Epoch: 0000; RMSE:1.271652020727891; MAE:1.2068166501602013
Epoch: 0003; RMSE:3.395880120080637; MAE:3.2778387187887823
Epoch: 0006; RMSE:3.019097771500957; MAE:2.9246250185822866
Epoch: 0009; RMSE:3.1281852073013403; MAE:3.0446889785747255
Epoch: 0012; RMSE:3.281345090150488; MAE:3.199182390928675
Epoch: 0015; RMSE:3.3126623980256578; MAE:3.233299925165594
Epoch: 0018; RMSE:3.3289505319424397; MAE:3.2503713419119524
Epoch: 0021; RMSE:3.349235526999219; MAE:3.2697282262048346
Epoch: 0024; RMSE:3.3673090763898306; MAE:3.287324126473677
Epoch: 0027; RMSE:3.3754217838125005; MAE:3.2941793084135074
Epoch: 0030; RMSE:3.3812906322876137; MAE:3.29910139633321
Epoch: 0033; RMSE:3.3841023713420033; MAE:3.3006195803621536
Epoch: 0036; RMSE:3.3899664293515674; MAE:3.3054805266094753
Epoch: 0039; RMSE:3.3879063818468205; MAE:3.3023662388440274
Epoch: 0042; RMSE:3.389086274646865; MAE:3.301968842345234
Epoch: 0045; RMSE:3.386

In [ ]:
errors_list